In [ ]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np

In [ ]:

base_url = f"https://www.gamedaycalendar.com/schedules/nba/{2023}"

teams = pd.read_html(base_url)
teams_df = pd.concat([division.melt() for division in teams])
teams_df.head()


In [ ]:
seasons = list(range(2014, 2024))
print(seasons)
data_frames = []
for season in seasons:
    # print(season)
    teams = pd.read_html(base_url)
    teams_df = pd.concat([division.melt() for division in teams])
    teams_df['season'] = season
    data_frames.append(teams_df)
# data_frames = [x for x in data_frames if str(type(x)).lower == ""]
division_df = pd.concat(data_frames)
print(division_df['value'].nunique())
print(division_df.shape)
division_df.head(2)

In [ ]:
# load team data to get the 3 letter abbreviations for each team
team_info_df = pd.read_csv("../data/team_info.csv")
abbreviations = team_info_df['abbreviation'].str.lower().tolist()
abbreviations[:5]

In [ ]:
# season 2023
url = "https://www.gamedaycalendar.com/schedules/nba/"
months = list(range(12))
dfs = []
errors = []
fixes = {'bkn':"brk", 'phx':"pho", 'sas':"san"}


for season in seasons:
    print(season)
    for team_abbr in abbreviations:
        team_abbr_mod = team_abbr
        if team_abbr_mod in ['bkn', 'phx', 'sas']:
            team_abbr_mod = fixes[team_abbr_mod]
        month_dfs = []
        for month in months:
            try:
                page_data = pd.read_html(
                    f"https://www.gamedaycalendar.com/schedules/nba/{team_abbr_mod}/{season}/{month}"
                )
            except:
                print(f"Error: {season}, {team_abbr}, {month}")
                errors.append(team_abbr)
                continue
            if len(page_data) >= 2:
                temp_df = page_data[1]
                temp_df["team_abbr"] = team_abbr
                temp_df['season'] = season
                month_dfs.append(temp_df)
        if len(month_dfs) > 0:
            dfs.append(pd.concat(month_dfs))

print("Done")
# print(set(errors))
# dfs

In [ ]:
game_time_df = pd.concat(dfs)
print(game_time_df.shape)
game_time_df.sample(5)

In [ ]:
# do some cleaning

# drop rows that say regular season
game_time_df = game_time_df[
    game_time_df['Date'] != "Regular Season"
].copy()

# split out the date
game_time_df[["day_of_week", "month", "day_of_month"]] = game_time_df['Date'].str.replace(",", "").str.split(" ", expand=True)


# get home vs away
game_time_df["is_home_game"] = np.where(game_time_df['Opponent'].str.contains("@"), 0, 1)

# clean up opponent
# game_time_df['Opponent'] = game_time_df['Opponent'].str.split(" ")

# game_time_df['time_time'] = pd.to_datetime(game_time_df['Time'])



print(game_time_df.shape)
game_time_df.head()

In [ ]:
# game_time_df['Opponent'].unique()

In [ ]:
# pd.read_html("https://www.gamedaycalendar.com/schedules/nba/bos/2014/7")#[1]


In [ ]:
"vs Altanta Hawks".split(" ", 1)